This notebook analyzes the files generated druing the sex check.

After generating general reports on the whole data, we also want to analyze the sex chromosomes separately. We use PLINK 1.9 software to determine the sex based on the data on the X chromosome (1=male, 2=female). The sex check algorithm flags individuals whom the reported sex in the PED file does not match the estimated sex (from given genomic data). 
The following commands were used to generate the final files.

plink --vcf PREFIX.chrX.vcf --keep-allele-order --vcf-half-call m --make-bed --out PREFIX.chrX.plink

plink --bfile PREFIX.chrX.plink --update-sex $new_sex --make-bed --out PREFIX.chrX.plink.sex

plink --bfile PREFIX.chrX.plink.sex --split-x hg38 --make-bed --out PREFIX.chrX.plink.sex.split-x

plink --bfile PREFIX.chrX.plink.sex.split-x --set-missing-var-ids @:#[b37]\\\\$1,\\\\$2 --make-bed --out PREFIX.chrX.plink.sex.split-x.id

plink --bfile PREFIX.chrX.plink.sex.split-x.id --indep-pairphase 20000 2000 0.5

plink --bfile PREFIX.chrX.plink.sex.split-x.id --extract plink.prune.in --make-bed --out PREFIX.chrX.plink.sex.split-x.id.LD

plink --bfile PREFIX.chrX.plink.sex.split-x.id.LD --check-sex --out PREFIX.chrX.plink.sex.split-x.id.LD.Xsex

At the end of the analysis we have multiple files:

1. joint_colombia_annotated.hg38.chrX.plink.sex.split-x.id.LD.bed

PLINK binary biallelic genotype table, primary representation of genotype calls at biallelic variants. Must be accompanied by .bim and .fam files.


2. joint_colombia_annotated.hg38.chrX.plink.sex.split-x.id.LD.bim

PLINK extended MAP file (each line of the MAP file describes a single marker) , extended variant information file accompanying a .bed binary genotype table.
Info includes: Chromosome code, Variant identifier, Position in morgans, Base-pair coordinate, allele 1, allele 2


3. joint_colombia_annotated.hg38.chrX.plink.sex.split-x.id.LD.fam

PLINK sample information file, information includes: Family ID ('FID'), Within-family ID ('IID'; cannot be '0'), Within-family ID of father , Within-family ID of mother, Sex code, Phenotype value 


4. joint_colombia_annotated.hg38.chrX.plink.sex.split-x.id.LD.Xsex.hh

heterozygous haploid and nonmale Y chromosome call list, produced automatically when the input data contains heterozygous calls where they shouldn't be possible (haploid chromosomes, male X/Y), or there are nonmissing calls for nonmales on the Y chromosome.


5. joint_colombia_annotated.hg38.chrX.plink.sex.split-x.id.LD.Xsex.log


6. joint_colombia_annotated.hg38.chrX.plink.sex.split-x.id.LD.Xsex.sexcheck

X chromosome-based sex validity report, info includes: Family ID, Within-family ID, Sex code in input file, Imputed sex code, Inbreeding coefficient, considering only X chromosome

In [1]:
# Import relevant libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
notebook_path = os.path.abspath("preQC_general_analysis.ipynb")
name = 'joint_colombia_annotated.hg38'
sexcheck_file_path = os.path.join(os.path.dirname(notebook_path), "data/chrX/" + name + ".chrX.plink.sex.split-x.id.LD.Xsex.sexcheck")


# Separate samples into XX and XY based on plink algorithms
samples_1, samples_2, samples_problem = [], [], []
ped_2 = 0
with open(sexcheck_file_path, 'r') as f:
    f.readline()     # skip header
    for line in f:
        sample_id, pedsex, snpsex = line.split()[0], line.split()[2], line.split()[3]
        if pedsex == '2':
            ped_2 += 1
        if snpsex == '2':
            samples_2.append(sample_id)
        elif snpsex == '1':
            samples_1.append(sample_id)
        else:
            samples_problem.append(sample_id)
            
            
print('1: %d, 2:%d, problems: %d' %(len(samples_1), len(samples_2), len(samples_problem)))
print('expected 2: %d' %ped_2)
print('total: %d' %len(samples_1+samples_2+samples_problem))

1: 53, 2:569, problems: 403
expected 2: 611
total: 1025
